# Análisis de datos utilizando PyRoot


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import ROOT
ROOT.gROOT.Reset()

In [3]:
#ROOT.gSystem.AddDynamicPath("/scratch/cms/MadGraph/MG5_aMC_v2_5_2/Delphes/")
ROOT.gSystem.AddDynamicPath("/scratch/cms/Delphes/delphes/")

In [4]:
ROOT.gSystem.Load("libDelphes");

In [ ]:
import os

In [ ]:
cwd = os.getcwd()
print cwd

In [ ]:
import commands

In [ ]:
tmp = commands.getstatusoutput('ls ../Datos/Signal_2j_mu-nu-Run0/Events')

In [ ]:
tmp[1]

In [ ]:
Runs = tmp[1].split("\n") 

In [ ]:
Runs

In [ ]:
MainChain=ROOT.TChain("Delphes")

In [ ]:
for run in range(len(Runs)):
    imput = "../Datos/Signal_2j_mu-nu-Run0/Events/"+Runs[run]+"/tag_1_delphes_events.root"
    MainChain.Add(imput)
    

In [ ]:
MainChain.Print()

In [ ]:
MainChain.Show()

In [ ]:
MainChain.GetEntry(1)

In [ ]:
t=ROOT.TBrowser()

In [ ]:
numberOfEntries = MainChain.GetEntries()

In [ ]:
numberOfEntries

In [ ]:
branchJet = MainChain.GetBranch("Jet")

In [ ]:
branchJet.


In [ ]:
pt=branchJet.GetLeaf("camilo")

In [ ]:
pt.GetValueLongDouble